In [1]:
import nltk
s1 = 'דיכאון'
s2 = 'דכאון'
nltk.edit_distance(s1, s2, transpositions=False)

1

In [2]:
typo_word= 'בטיחון'

In [3]:
##Creating a list of stop words and adding custom stopwords
import codecs
with codecs.open('bgulex.utf8.hr', 'r', encoding='utf8') as f:
    text = f.read()

In [4]:
lines = text.split('\n')

In [5]:
words_in_heb_dict = set()
for line in lines:
    word = line.split(':')[0].split(' ')[0].strip().replace(' DEF','')
    if len(word) > 0:
        words_in_heb_dict.add(word)

In [6]:
nearest_words_in_keyboard = {'א' : 'רכעיט',
'ב' : 'סדגכה',
'ג' : 'רכהבסד',
'ד' : 'גבסזש',
'ה' : 'בגכענ',
'ו' : 'םךלחו',
'ז' : 'סדש',
'ח' : 'יטוןלתצמ',
'ט' : 'אעחו',
'י' : 'אטוחצמנע',
'כ' : 'קראענהבג',
'ל' : 'חוןםךצ',
'מ' : 'ניחצ',
'נ' : 'מיעה',
'ס' : 'זשדגב',
'ע' : 'כראטינה',
'פ' : 'םלך',
'צ' : 'מחלת',
'ק' : 'רכגד',
'ר' : 'קגכעא',
'ש' : 'דסז',
'ת' : 'צלךץ',
'ם' : 'ןלךףפ',
'ן' : 'וחלךם',
'ץ' : 'ףךת',
'ף' : 'פםךץ',
'ך' : 'ףפםןלת'}

In [41]:
def generate_words_that_has_nearest_letters_in_keyboard(nearest_words_in_keyboard,word,number_letters=1):
    closest_words = list()
    for letter_index in range(len(word)):
        letter = word[letter_index]
        for near_letters in nearest_words_in_keyboard[letter]:
            new_word = str(word[:letter_index] + near_letters +word[letter_index+1:])
            closest_words.append(new_word)
            if number_letters > 1:
                for new_word_ in fix_by_nearest_letters_in_keyboard(nearest_words_in_keyboard,new_word,number_letters-1):
                    closest_words.append(new_word_)
    for close_word in set(closest_words):
        yield close_word

In [8]:
def permutations(string,number_swaps=1,location=0):
    # everything to the right of step has not been swapped yet
    prem_words = []
    for i in range(location,len(string)-1):

        # copy the string (store as array)
        string_copy = [character for character in string]

        # swap the current index with the step
        string_copy[i], string_copy[i+1] = string_copy[i+1], string_copy[i]

        prem_words += [''.join(string_copy)]
        if number_swaps > 1:
            for i in permutations(string_copy,number_swaps-1,location+3):
                prem_words += [i]
    for i in set(prem_words):
        yield i


In [9]:
def generate_transpory_words(word,heb_dict):
    perms = [''.join(p) for p in permutations(word)]
    for perm in perms:
        if perm in heb_dict:
            yield perm

In [10]:
for i in generate_transpory_words(typo_word,words_in_heb_dict):
    print(i)

ביטחון


In [31]:
for i in fix_by_nearest_letters_in_keyboard(nearest_words_in_keyboard,typo_word):
    if i in words_in_heb_dict:
        print(i)

בטיחוך
בטיוון
הטיחון
בטצחון
בטיחול
כטיחון
בטיצון
סטיחון
באיחון
בטיחךן
בחיחון
בטנחון
בטיחון
בטיחוו
בטטחון
בטחחון
בטיחום
בטעחון
בטאחון
דטיחון
בטיןון
בטייון
בטיחםן
גטיחון
בטיחחן
בטמחון
בויחון
בטיטון
בטוחון
בטילון
בטיתון
בטיחוח
בטיחלן
בעיחון
בטימון


In [12]:
def find_closest_words(heb_dict,distance_word):
    distance = -1
    closest_words = list()
    for word in heb_dict:
        curr_distance = nltk.edit_distance(word, distance_word, transpositions=True)
        if distance == curr_distance:
            closest_words += [word]
        if distance > curr_distance or distance == -1:
            distance = curr_distance
            closest_words = [word]
    closest_words = sorted(closest_words, key=len)
    return closest_words, distance

In [53]:
def common_typo_hebrew(word, start_index=0):
    common_words = list()
    for letter_index in range(start_index, len(word)):
        letter = word[letter_index]
        near_letters = [letter]
        letter = word[letter_index]
        if letter == 'כ':
            near_letters = ['ח','ק']
        if letter == 'ח':
            near_letters = ['ק','כ']
        if letter == 'ק':
            near_letters = ['ח','כ']
        if letter == 'ש':
            near_letters = ['ס']
        if letter == 'ס':
            near_letters = ['ש']
        if letter == 'ו':
            near_letters = ['ב']
        if letter == 'ב':
            near_letters = ['ו']
        if letter == 'א':
            near_letters = ['ע','ה']
        if letter == 'ה':
            near_letters = ['ע','א']
        if letter == 'ע':
            near_letters = ['א','ה']
        if letter == 'ט':
            near_letters = ['ת']
        if letter == 'ת':
            near_letters = ['צ']
        if near_letters[0] != letter:
            for near_letter in near_letters:
                new_word = str(word[:letter_index] + near_letter +word[letter_index+1:])
                common_words += [new_word]
                common_words += common_typo_hebrew(new_word,letter_index+1)
    for common_word in common_words:
        yield common_word

In [13]:
%time find_closest_words(words_in_heb_dict, typo_word)

CPU times: user 27.7 s, sys: 93.8 ms, total: 27.8 s
Wall time: 29.3 s


(['בטיחן', 'בטחון', 'בטיחו', 'ביטחון', 'בטיחות', 'בטיחכן', 'בטיחותן'], 1)

In [22]:
problem_words = list()
##Creating a list of stop words and adding custom stopwords
import codecs
with codecs.open('problematic_words.txt', 'r', encoding='utf8') as f:
    text = f.read()
for word in text.split('\n'):
    problem_words.append(word.strip())

In [56]:
for word in set(problem_words):
    for i in generate_transpory_words(word,words_in_heb_dict):
        print(f'{word} ------> {i}')

טיפ ------> טפי
למכים ------> מלכים
ערקון ------> עקרון
המשקיות ------> המשיקות
השחמ ------> השמח
סמחט ------> מסחט
המחוה ------> המוחה
וצהל ------> צוהל
מלשב ------> משלב
חווד ------> חודו
פקא ------> קפא
הסמחט ------> המסחט
פקל ------> קפל
אלמ ------> אמל
קורל ------> קרול
קורל ------> קולר
יא ------> אי
מפלחץ ------> מפחלץ
ותש ------> תוש
גנשים ------> נגשים
עוב ------> עבו
מודעיין ------> מודיעין
נשמ ------> נמש
בסדיר ------> בסדרי
באיוש ------> באישו
רמח ------> מרח
שאלך ------> אשלך
יב ------> בי
תיהיה ------> תהייה
תאגדים ------> תאגידם
הצלטמתי ------> הצטלמתי
משבוע ------> משובע
מדירכת ------> מדריכת
במזי ------> מבזי
ההיטס ------> ההיסט
לגעת ------> לעגת
תתפקיד ------> תתפקדי
המשקית ------> המקשית
מדאגים ------> מדאיגם
מלשבים ------> משלבים
קמני ------> מקני
הדינות ------> הדניות
החא ------> האח
הגביוה ------> הגבויה
הגביוה ------> הגביהו
באוק ------> באקו
גמילים ------> גימלים
הדחש ------> החדש
המלשב ------> המשלב
מחיפה ------> מחפיה
הקמית ------> הקמתי
החטל ------> החלט
ימני

In [55]:
for word in set(problem_words):
    #for i in generate_transpory_words(word,words_in_heb_dict):
    for i in common_typo_hebrew(word):
        if i in words_in_heb_dict:
            print(f'{word} ------> {i}')

היווי ------> עיבוי
היווי ------> איווי
ערקון ------> הרכבן
הסטילים ------> השתילים
המשקיות ------> המשכיות
אחרכן ------> עקרכן
אחרכן ------> הכרחן
הטסט ------> התשת
הטסט ------> הטסת
סמחט ------> שמחת
סמחט ------> סמקת
סמחט ------> סמכת
אסמכתה ------> הסמכתה
אסמכתה ------> הסמקתה
המחוה ------> המקבע
המחוה ------> המכבה
בבהד ------> וועד
רפועיות ------> רפואיות
לוקר ------> לבקר
לוקר ------> לבכר
חווד ------> כבוד
חווד ------> כובד
פקא ------> פחה
פקא ------> פכה
פקא ------> פקע
אבטש ------> הותש
אבטש ------> הוטס
אבטש ------> אותש
אבטש ------> אוטס
יטם ------> יתם
האכא ------> אאחה
האכא ------> העקה
הסמחט ------> הסמקת
הסמחט ------> הסמכת
כמכ ------> חמק
כמכ ------> קמח
התים ------> עתים
התים ------> עצים
התים ------> אתים
התים ------> אצים
עוב ------> אוב
עוב ------> אבו
עוב ------> אוו
עוב ------> הוו
עוב ------> עבו
עוב ------> עוו
תצו ------> צצו
מטכני ------> מתכני
מטכני ------> מתקני
פוט ------> פות
כפצת ------> חפצת
כפצת ------> קפצת
תגב ------> צגו
תגב ------> תגו
עתודב ------

KeyboardInterrupt: 